In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import lxml
import html5lib

Now that necessary packages are installed, going to parse the needed url. This URL brings us to the up to date team page for the St. Louis Blues . So I am going to scrape the cap friendly website (Team tab, STL Blues) and get this into a couple different dataframes (Draft, Forwards, Defense, Goalies, Season Opening Injured Reserve) that I can download and put into the MySQL database.

In [34]:
url_v3 = "https://www.capfriendly.com/teams/blues"

req2 = requests.get(url_v3)
soup = BeautifulSoup(req2.content, 'lxml')  # make a soup of html & css from the web page

In [51]:
#Find all tables
table = soup.find_all('table', class_='cf_teamProfileRosterSection__table cf_table__collapsiblePlayerColumn cf_table__default cf_table__fixed cf_table__stickyFirstColumn')

In [18]:
print(table)

[<table class="cf_teamProfileRosterSection__table cf_table__collapsiblePlayerColumn cf_table__default cf_table__fixed cf_table__stickyFirstColumn"><thead><tr><th>Forwards (10 - $40,982,262)</th><th class="tm_yrr hide" style="width:80px">Years Remaining</th><th>Terms</th><th>POS</th><th>Status</th><th>Acquired</th><th>Age</th><th class="q" title="Percentage of the current salary cap">Cap%</th><th>2024-25</th><th>2025-26</th><th>2026-27</th><th>2027-28</th><th>2028-29</th><th>2029-30</th></tr></thead><tbody><tr><td><a href="/players/jordan-kyrou">Kyrou, Jordan</a></td><td class="tm_yrr hide">7 UFA</td><td><span class="con_d"></span></td><td><span class="con_d">RW</span></td><td class="">NHL</td><td><a href="/transactions/players/jordan-kyrou"><span class="q" title="2016 Round 2, #35 Overall">Draft</span></a></td><td><span class="q" title="May 5, 1998">26</span></td><td>9.2</td><td style="position:rel"><span class="cap rel mtv">$8,125,000</span><span class="aav rel mtv hide">$8,125,000</s

In [52]:
forwards_df = pd.read_html(str(table),header=0, index_col = 0, na_values=["-"])[0]

C:\Users\nsofi\AppData\Local\Temp\ipykernel_33384\1387399994.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  forwards_df = pd.read_html(str(table),header=0, index_col = 0, na_values=["-"])[0]


In [49]:
print(forwards_df.columns)

Index(['Years Remaining', 'Terms', 'POS', 'Status', 'Acquired', 'Age', 'Cap%',
       '2024-25', '2025-26', '2026-27', '2027-28', '2028-29', '2029-30'],
      dtype='object')


Everything looks good and just need to clean the year columns where their cap hits are listed as in the df, it puts all their yearly cap hit inside each cell which is not what we want. So gonna create a function to apply on the dataframe for each year, and then apply the respective functions to the 2024-24, 2025-26, etc yearly header columns.

In [53]:
def conditional_splitx(value):
    if isinstance(value, str) and value.startswith('$'):
        # Split the value by '$' and keep the first part along with the first '$'
        first_dollar_index = value.find('$', 1)
        return value[:first_dollar_index]
    return value

# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    forwards_df[column] = forwards_df[column].apply(conditional_splitx)


# Apply the function and update the '2024-25' column
#forwards_df['2024-25'] = forwards_df['2024-25'].apply(conditional_splitx)

In [54]:
forwards_df.to_csv('Cap Friendly Blues Forwards.csv')


Forwards are scraped. Now time to scrape defensemen.

In [71]:
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
['cf_teamProfile__draftTable', 'cf_table__default', 'cf_table__fixed', 'cf_table__stickyFirstColumn']
['tbl', 'fixed']
['tbl', 'fixed']
['cf_teamProfileRosterSection__table', 'cf_table__collapsiblePlayerColumn', 'cf_table__default', 'cf_table__fixed', 'cf_table__stickyFirstColumn']
['cf_teamProfileRosterSection__table', 'cf_table__collapsiblePlayerColumn', 'cf_table__default', 'cf_table__fixed', 'cf_table__stickyFirstColumn']
['cf_teamProfileRosterSection__table', 'cf_table__collapsiblePlayerColumn', 'cf_table__default', 'cf_table__fixed', 'cf_table__stickyFirstColumn']
['cf_teamProfileRosterSection__table', 'cf_table__collapsiblePlayerColumn', 'cf_table__default', 'cf_table__fixed', 'cf_table__stickyFirstColumn']
['cf_teamProfileStats__table', 'cf_table__collapsiblePlayerColumn', 'cf_table__fixed', 'rel']
['cf_teamProfileStats__table', 'cf_table__collapsiblePlayerColumn', 'cf_table__fixed', 'rel']
['cf_teamProfileStats__table', 'cf_table__collapsiblePlayerColumn

In [77]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    defensemen_table = tables[2]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
df_defensemen = pd.read_html(str(defensemen_table))[0]

C:\Users\nsofi\AppData\Local\Temp\ipykernel_33384\460404493.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_defensemen = pd.read_html(str(defensemen_table))[0]


In [78]:
print(df_defensemen)

  Defense (6 - $25,450,000) Years Remaining Terms    POS Status Acquired  \
0         Faulk, Justin "A"           3 UFA   NTC     RD    NHL    Trade   
1               Krug, Torey           3 UFA   NTC     LD    NHL   Signed   
2       Parayko, Colton "A"           6 UFA   NTC     RD    NHL    Draft   
3               Leddy, Nick           2 UFA   NTC     LD    NHL   Signed   
4         Perunovich, Scott           1 RFA   NaN     LD    NHL    Draft   
5             Tucker, Tyler           1 RFA   NaN  LD/RD    NHL    Draft   
6          Scandella, Marco           0 UFA   NaN     LD    NHL    Trade   
7                     TOTAL             NaN   NaN    NaN    NaN      NaN   

    Age  Cap%                                           2024-25  \
0  32.0   7.4   $6,500,000$6,500,000$4,750,000$4,750,000$0$0NTC   
1  33.0   7.4   $6,500,000$6,500,000$8,500,000$8,500,000$0$0NTC   
2  31.0   7.4   $6,500,000$6,500,000$8,000,000$8,000,000$0$0NTC   
3  33.0   4.5   $4,000,000$4,000,000$3,500,000$

In [79]:
df_defensemen.head()

,"Defense (6 - $25,450,000)",Years Remaining,Terms,POS,Status,Acquired,Age,Cap%,2024-25,2025-26,2026-27,2027-28,2028-29,2029-30
0,"Faulk, Justin ""A""",3 UFA,NTC,RD,NHL,Trade,32.0,7.4,"$6,500,000$6,500,000$4,750,000$4,750,000$0$0NTC","$6,500,000$6,500,000$4,500,000$4,500,000$0$0M-NTC","$6,500,000$6,500,000$4,500,000$4,500,000$0$0M-NTC",UFA,NaN,NaN
1,"Krug, Torey",3 UFA,NTC,LD,NHL,Signed,33.0,7.4,"$6,500,000$6,500,000$8,500,000$8,500,000$0$0NTC","$6,500,000$6,500,000$6,500,000$6,500,000$0$0M-NTC","$6,500,000$6,500,000$6,000,000$6,000,000$0$0M-NTC",UFA,NaN,NaN
2,"Parayko, Colton ""A""",6 UFA,NTC,RD,NHL,Draft,31.0,7.4,"$6,500,000$6,500,000$8,000,000$8,000,000$0$0NTC","$6,500,000$6,500,000$8,000,000$8,000,000$0$0NTC","$6,500,000$6,500,000$6,350,000$6,350,000$0$0NTC","$6,500,000$6,500,000$4,800,000$4,800,000$0$0NTC","$6,500,000$6,500,000$4,800,000$4,800,000$0$0M-NTC","$6,500,000$6,500,000$4,800,000$4,800,000$0$0M-NTC"
3,"Leddy, Nick",2 UFA,NTC,LD,NHL,Signed,33.0,4.5,"$4,000,000$4,000,000$3,500,000$3,500,000$0$0NTC","$4,000,000$4,000,000$3,000,000$3,000,000$0$0M-NTC",UFA,NaN,NaN,NaN
4,"Perunovich, Scott",1 RFA,NaN,LD,NHL,Draft,25.0,1.3,"$1,150,000$1,150,000$1,150,000$1,150,000$0$0",RFA,NaN,NaN,NaN,NaN


In [80]:
#Function to clean the defense table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    df_defensemen[column] = df_defensemen[column].apply(conditional_splitx)

In [82]:
df_defensemen.to_csv('Cap Friendly Blues Defense.csv')

Now that we have forwards and defense, time for goalies

In [83]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    goalies_table = tables[3]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
goalies_df = pd.read_html(str(goalies_table))[0]

C:\Users\nsofi\AppData\Local\Temp\ipykernel_33384\273164630.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalies_df = pd.read_html(str(goalies_table))[0]


In [85]:
#Function to clean the goalies table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    goalies_df[column] = goalies_df[column].apply(conditional_splitx)

In [86]:
goalies_df.to_csv('Cap Friendly Blues Goalies.csv')

Forwards, Defense, and Goalies Scraped. Now time for the Season Opening Reserve

In [87]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    seasonOpenIR_table = tables[4]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
seasonOpenIR_df = pd.read_html(str(seasonOpenIR_table))[0]

C:\Users\nsofi\AppData\Local\Temp\ipykernel_33384\3016402215.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  seasonOpenIR_df = pd.read_html(str(seasonOpenIR_table))[0]


In [89]:
#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    seasonOpenIR_df[column] = seasonOpenIR_df[column].apply(conditional_splitx)

In [90]:
seasonOpenIR_df.to_csv('Cap Friendly Blues Season Opening IR.csv')

Non-Roster Time. First up, Forwards

In [93]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    forwardsNR_table = tables[8]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
forwardsNR_df = pd.read_html(str(forwardsNR_table))[0]

C:\Users\nsofi\AppData\Local\Temp\ipykernel_33384\2126343547.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  forwardsNR_df = pd.read_html(str(forwardsNR_table))[0]


In [95]:
#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    forwardsNR_df[column] = forwardsNR_df[column].apply(conditional_splitx)

In [96]:
forwardsNR_df.to_csv('Cap Friendly Blues Non-Roster Forwards.csv')

Non-Roster Defense

In [97]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    defenseNR_table = tables[9]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
defenseNR_df = pd.read_html(str(defenseNR_table))[0]

#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    defenseNR_df[column] = defenseNR_df[column].apply(conditional_splitx)

defenseNR_df.to_csv('Cap Friendly Blues Non-Roster Defense.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_33384\2771656481.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defenseNR_df = pd.read_html(str(defenseNR_table))[0]


Non-Roster Goalies

In [98]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    goaliesNR_table = tables[10]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
goaliesNR_df = pd.read_html(str(goaliesNR_table))[0]

#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    goaliesNR_df[column] = goaliesNR_df[column].apply(conditional_splitx)

goaliesNR_df.to_csv('Cap Friendly Blues Non-Roster Goalies.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_33384\721026541.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goaliesNR_df = pd.read_html(str(goaliesNR_table))[0]
